<a href="https://colab.research.google.com/github/piggythefish/deep_reinforcement_learning/blob/main/tf_hw_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds
import tqdm


shuffle_buffer_size = 1000
batch_size = 64
epochs = 100
lr = 1e-3

def load_prep_cifar(batch_size, shuffle_buffer_size):
    train = tfds.load("cifar10", split="train", as_supervised=True)

    def preprocessing_func(img, label):
        img = tf.cast(img, tf.float32)
        img = (img / 128) - 1
        label = tf.one_hot(label, depth=10)
        return img, label

    train = train.map(lambda img, label: preprocessing_func(img, label))
    train = train.shuffle(1000).batch(batch_size).prefetch(tf.data.AUTOTUNE)
    return train


train = load_prep_cifar(batch_size, shuffle_buffer_size)


def create_my_cifar_cnn(name="nice_cnn"):
    inputs = tf.keras.Input(shape=(32, 32, 3), dtype=tf.float32)
    cnn_layer_1 = tf.keras.layers.Conv2D(
        filters=32, kernel_size=3, activation="relu", padding="same"
    )
    x = cnn_layer_1(inputs)  # shape: batch_size, 32, 32, 5
    cnn_layer_2 = tf.keras.layers.Conv2D(
        filters=64, kernel_size=3, activation="relu", padding="same"
    )
    x = cnn_layer_2(x)  # shape: batch_size, 32, 32, 10
    cnn_layer_3 = tf.keras.layers.Conv2D(
        filters=128, kernel_size=3, activation="relu", padding="same"
    )
    x = cnn_layer_3(x)  # shape: batch_size, 32, 32, 64
    global_pooling = tf.keras.layers.GlobalAveragePooling2D()
    x = global_pooling(x)  # shape: batch_size, 64
    output_layer = tf.keras.layers.Dense(10, activation="softmax")
    output = output_layer(x)
    model = tf.keras.Model(inputs, output, name=name)
    return model

def test(model, batch_size):
    test_data = load_prep_cifar(batch_size=batch_size, shuffle_buffer_size=shuffle_buffer_size, split="test")

    loss_f = tf.keras.losses.CategoricalCrossentropy()
    acc = tf.keras.metrics.CategoricalAccuracy()

    losses = tf.keras.metrics.Mean()
    acc.reset_states()

    for x, t in test_data:
        pred = model(x)
        l = loss_f(t, pred)
        losses.update_state(l)
        acc.update_state(t, pred)

    print(f"Test - Loss: {float(losses.result()):.4f} - Accuracy: {float(acc.result()):.4f}")
    return float(losses.result()), float(acc.result())



def train(epochs, batch_size, shuffle_buffer_size, lr):
    model = create_my_cifar_cnn()
    print(model.summary())

    data = load_prep_cifar(
        batch_size=batch_size, shuffle_buffer_size=shuffle_buffer_size
    )
    optimizer = tf.keras.optimizers.legacy.Adam(learning_rate=lr)
    loss_f = tf.keras.losses.CategoricalCrossentropy()

    losses = tf.keras.metrics.Mean()
    acc = tf.keras.metrics.CategoricalAccuracy()

    for epoch in range(epochs):
        bar = tqdm.tqdm(data, desc=f"Epoch {epoch:2d}")

        for x, t in bar:
            with tf.GradientTape() as tape:
                pred = model(x)
                l = loss_f(t, pred)
            gradients = tape.gradient(l, model.trainable_variables)
            optimizer.apply_gradients(zip(gradients, model.trainable_variables))

            losses.update_state(l)
            acc.update_state(t, pred)

            bar.set_description(
                f"Epoch {epoch:2d} - Loss {float(losses.result()):.4f} - Accuracy {float(acc.result()):.4f}"
            )



model = train(
    epochs=epochs, batch_size=batch_size, shuffle_buffer_size=shuffle_buffer_size, lr=lr
)

# Test the model
test(model, batch_size)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/2 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/50000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/cifar10/3.0.2.incomplete0U3CH4/cifar10-train.tfrecord*...:   0%|          …

Generating test examples...:   0%|          | 0/10000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/cifar10/3.0.2.incomplete0U3CH4/cifar10-test.tfrecord*...:   0%|          |…

Dataset cifar10 downloaded and prepared to /root/tensorflow_datasets/cifar10/3.0.2. Subsequent calls will reuse this data.
Model: "nice_cnn"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 32, 32, 3)]       0         
                                                                 
 conv2d (Conv2D)             (None, 32, 32, 32)        896       
                                                                 
 conv2d_1 (Conv2D)           (None, 32, 32, 64)        18496     
                                                                 
 conv2d_2 (Conv2D)           (None, 32, 32, 128)       73856     
                                                                 
 global_average_pooling2d (  (None, 128)               0         
 GlobalAveragePooling2D)                                         
                                                                 
 

Epoch 10 - Loss 1.3341 - Accuracy 0.5227:  19%|█▊        | 145/782 [00:05<00:23, 27.07it/s]

In [ ]:
import matplotlib.pyplot as plt


In [ ]:
epoch_losses = []
epoch_accuracies = []


In [ ]:
epoch_losses.append(float(losses.result()))
epoch_accuracies.append(float(acc.result()))


In [ ]:
def plot_metrics(epoch_losses, epoch_accuracies):
    epochs_range = range(len(epoch_losses))

    plt.figure(figsize=(12, 4))

    plt.subplot(1, 2, 1)
    plt.plot(epochs_range, epoch_losses, label='Loss')
    plt.title('Training Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()

    plt.subplot(1, 2, 2)
    plt.plot(epochs_range, epoch_accuracies, label='Accuracy')
    plt.title('Training Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()

    plt.show()


In [ ]:
plot_metrics(epoch_losses, epoch_accuracies)


In [ ]:
test(model, batch_size)

In [ ]:
test_loss, test_accuracy = test(model, batch_size)
plt.bar(['Loss', 'Accuracy'], [test_loss, test_accuracy])
plt.title('Test Performance')
plt.show()